In [ ]:
#Updated pip for Digital Ocean
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
!pip install transformers==4.53.1 datasets==4.0.0 accelerate==1.8.1 pandas==2.2.2 plotly==6.1.1 huggingface_hub==0.33.2 tqdm==4.67.1 scikit-learn==1.6.1 bitsandbytes kaleido>=0.1.0
!pip install matplotlib seaborn

In [ ]:
# Simple HuggingFace Login
from huggingface_hub import login, whoami
import os

# Set your token
hf_token = "HF_Secret"

# Set environment variable
os.environ['HF_TOKEN'] = hf_token

try:
    # Login to HuggingFace
    login(hf_token, add_to_git_credential=True)
    print("✅ Successfully logged in to HuggingFace!")
    
    # Verify login
    user_info = whoami()
    print(f"✅ Logged in as: {user_info['name']}")
    
except Exception as e:
    print(f"❌ Login failed: {e}")
    print("Please check your token and try again.")

In [ ]:
# -*- coding: utf-8 -*-
"""
This script merges the functionalities of fine-tuning a language model and performing
layer-wise bias analysis, including the final step of uploading the model to the
Hugging Face Hub. It is designed to be run on platforms like Google Colab or
DigitalOcean.

Workflow:
1.  **Initial Bias Analysis**: Performs a layer-wise bias analysis on the base
    model (`EleutherAI/pythia-70m`) using an English dataset before any fine-tuning.
2.  **Fine-tuning**: Fine-tunes the `EleutherAI/pythia-70m` model on a Hindi dataset
    (`iamshnoo/alpaca-cleaned-hindi`).
3.  **Post-Epoch Bias Analysis**: After each fine-tuning epoch, the script runs the
    layer-wise bias analysis on the model for both English and Hindi to track how
    bias evolves.
4.  **Results**: All bias analysis results are saved to CSV files for further examination.
5.  **Upload**: After training, the script can be run again with the 'upload' action
    to push the final model, tokenizer, and training artifacts to the Hugging Face Hub.

This script can be executed with command-line arguments to specify the action
('train' or 'upload'), training mode ('test' or 'full'), and the platform
('colab' or 'local').
"""

# In a notebook environment (like Colab), run this cell first to install dependencies.
# !pip install -q transformers>=4.32.0 datasets torch pandas numpy scikit-learn accelerate bitsandbytes

import os
import gc
import sys
import json
import csv
import argparse
import warnings
from tqdm import tqdm
import torch
import pandas as pd
import numpy as np
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    TrainerCallback,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from datasets import load_dataset
from huggingface_hub import HfApi, login
from sklearn.metrics.pairwise import cosine_similarity

# =============================================================================
# GLOBAL CONFIGURATION
# =============================================================================
BASE_MODEL_NAME = "EleutherAI/pythia-70m"
DATASET_NAME = "iamshnoo/alpaca-cleaned-hindi"
HF_USERNAME = "DebK"  # <-- IMPORTANT: SET YOUR USERNAME HERE
NEW_MODEL_REPO_NAME = f"{HF_USERNAME}/pythia-70m-finetuned-alpaca-hindi"
FINAL_MODEL_DIR = "./pythia-70m-hindi-final"
BIAS_RESULTS_DIR = "./bias_analysis_results"

# =============================================================================
# PLATFORM-SPECIFIC CONFIGURATION
# =============================================================================

def setup_platform_environment(platform: str = "local"):
    """
    Configures the environment based on the specified platform.

    Args:
        platform (str): The target platform, e.g., 'colab', 'digitalocean', 'local'.

    Returns:
        tuple: A tuple containing (project_path, results_path, hf_cache_dir).
    """
    print(f"Setting up environment for: {platform.upper()}")

    if platform == "colab":
        try:
            from google.colab import drive, userdata
            drive.mount('/content/drive')
            print("Google Drive mounted successfully.")

            project_path = "/content/drive/MyDrive/Mult_LLM_Bias/"
            results_path = os.path.join(project_path, "Results/Experiment1/")
            hf_cache_dir = os.path.join(project_path, "hf_cache/")

            print("Attempting to log in to Hugging Face...")
            HF_TOKEN = userdata.get('HF_TOKEN')
            if HF_TOKEN:
                login(HF_TOKEN, add_to_git_credential=True)
                print("Successfully logged in to Hugging Face!")
            else:
                print("Hugging Face token not found in Colab secrets.")

        except ImportError:
            print("Could not import Google Colab libraries. Defaulting to local setup.")
            return setup_platform_environment("local")

    elif platform == "digitalocean":
        print("Setting up DigitalOcean environment...")
        
        # Set paths for DigitalOcean - using root directory as base
        project_path = "/root/Mult_LLM_Bias/"
        results_path = os.path.join(project_path, "Results/Experiment1/")
        hf_cache_dir = os.path.join(project_path, "hf_cache/")
        
        # Check if block storage volume is available (optional)
        volume_path = "/mnt/volume_nyc1_01/"
        if os.path.exists(volume_path):
            print("DigitalOcean volume detected. Using volume storage...")
            project_path = os.path.join(volume_path, "Mult_LLM_Bias/")
            results_path = os.path.join(project_path, "Results/Experiment1/")
            hf_cache_dir = os.path.join(project_path, "hf_cache/")
        else:
            print("Using root directory storage...")
        
        # Set up Hugging Face authentication for DigitalOcean
        print("Setting up Hugging Face authentication...")
        
        # First try environment variable
        hf_token = os.environ.get('HF_TOKEN')
        if not hf_token:
            # Set your token
            hf_token = "HF_Secret"
            # Set it as environment variable for this session
            os.environ['HF_TOKEN'] = hf_token
        
        try:
            login(hf_token, add_to_git_credential=True)
            print("✅ Successfully logged in to Hugging Face!")
        except Exception as e:
            print(f"❌ Failed to login to Hugging Face: {e}")
            print("Please check your token and internet connection.")
        
        # Optimize for DigitalOcean environment
        os.environ['TRANSFORMERS_CACHE'] = hf_cache_dir
        os.environ['HF_HOME'] = hf_cache_dir
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Use first GPU if available
        
        print("DigitalOcean environment configured successfully.")

    else: # Default to local setup
        project_path = "./"
        results_path = BIAS_RESULTS_DIR
        hf_cache_dir = "./hf_cache/"

    os.makedirs(results_path, exist_ok=True)
    os.makedirs(hf_cache_dir, exist_ok=True)

    print(f"Project path set to: {project_path}")
    print(f"Bias analysis results will be saved to: {results_path}")

    return project_path, results_path, hf_cache_dir


# =============================================================================
# LAYER-WISE BIAS ANALYSIS COMPONENTS
# =============================================================================

class LLMManager:
    """Manages the lifecycle of LLMs to optimize memory usage."""
    def __init__(self, cache_dir: str):
        self.cache_dir = cache_dir
        self.model = None
        self.tokenizer = None
        self.current_model_id = None

    def load_model(self, model_id: str, model_repo: str):
        """Load model and tokenizer from either Hugging Face or a local path."""
        if self.current_model_id == model_id and self.model is not None:
            print(f"Model '{model_id}' already loaded.")
            return self.model, self.tokenizer

        print(f"Loading model: {model_id} from {model_repo}")
        load_path = model_id if model_repo == 'hf' else model_repo

        try:
            self.tokenizer = AutoTokenizer.from_pretrained(load_path, cache_dir=self.cache_dir)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            #Update Config
            quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16
             )

            # quantization_config = BitsAndBytesConfig(load_in_4bit=True)
            self.model = AutoModelForCausalLM.from_pretrained(
                load_path,
                torch_dtype=torch.float16,
                device_map="auto",
                quantization_config=quantization_config,
                cache_dir=self.cache_dir
            )
            self.current_model_id = model_id
            print(f"Model '{model_id}' loaded successfully.")
            return self.model, self.tokenizer

        except Exception as e:
            print(f"ERROR: Failed to load model '{model_id}'. Exception: {e}")
            return None, None

    def unload_model(self):
        """Unloads the model and clears GPU cache."""
        if self.model:
            print(f"Unloading model: {self.current_model_id}...")
            del self.model
            del self.tokenizer
            self.model, self.tokenizer, self.current_model_id = None, None, None
            gc.collect()
            torch.cuda.empty_cache()
            print("Model unloaded and memory cleared.")

class WEATHubLoader:
    """Loads the WEATHub dataset and provides word lists."""
    def __init__(self, dataset_id: str, cache_dir: str = None):
        print(f"Loading WEATHub dataset from '{dataset_id}'...")
        try:
            self.dataset = load_dataset(dataset_id, cache_dir=cache_dir)
            print("WEATHub dataset loaded successfully.")
            self.split_mapping = {
                'WEAT1': 'original_weat', 'WEAT2': 'original_weat', 'WEAT6': 'original_weat', 'WEAT7': 'original_weat', 'WEAT8': 'original_weat'
            }
        except Exception as e:
            print(f"ERROR: Failed to load WEATHub dataset. Exception: {e}")
            self.dataset = None

    def get_word_lists(self, language_code: str, weat_category_id: str):
        """Retrieves target and attribute word lists."""
        if not self.dataset: return None
        split_name = self.split_mapping.get(weat_category_id)
        if not split_name:
            print(f"Warning: Category '{weat_category_id}' not found.")
            return None
        try:
            filtered = self.dataset[split_name].filter(lambda x: x['language'] == language_code and x['weat'] == weat_category_id)
            if len(filtered) > 0:
                return { 'targ1': filtered[0]['targ1.examples'], 'targ2': filtered[0]['targ2.examples'], 'attr1': filtered[0]['attr1.examples'], 'attr2': filtered[0]['attr2.examples'] }
            else:
                print(f"Warning: No data for language '{language_code}' and category '{weat_category_id}'.")
                return None
        except Exception as e:
            print(f"Error filtering data for '{weat_category_id}' in language '{language_code}': {e}")
            return None

class LayerEmbeddingExtractor:
    """Extracts hidden states from model layers."""
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.device = model.device

    @torch.no_grad()
    def get_embeddings(self, words: list, layer_idx: int):
        """Gets embeddings for a list of words at a specific layer."""
        all_embeddings = []
        for word in words:
            inputs = self.tokenizer(word, return_tensors="pt", add_special_tokens=False).to(self.device)
            outputs = self.model(**inputs, output_hidden_states=True)
            word_embedding = outputs.hidden_states[layer_idx][0].mean(dim=0).float().cpu().numpy()
            all_embeddings.append(word_embedding)
        return np.array(all_embeddings)

class BiasQuantifier:
    """Calculates bias scores using WEAT effect size."""
    def _s(self, w, A, B):
        mean_cos_A = np.mean([cosine_similarity([w], [a])[0][0] for a in A])
        mean_cos_B = np.mean([cosine_similarity([w], [b])[0][0] for b in B])
        return mean_cos_A - mean_cos_B

    def weat_effect_size(self, T1_embeds, T2_embeds, A1_embeds, A2_embeds):
        """Calculates the WEAT effect size (d-score)."""
        mean_T1 = np.mean([self._s(t, A1_embeds, A2_embeds) for t in T1_embeds])
        mean_T2 = np.mean([self._s(t, A1_embeds, A2_embeds) for t in T2_embeds])
        all_s = [self._s(t, A1_embeds, A2_embeds) for t in np.concatenate((T1_embeds, T2_embeds))]
        std_dev = np.std(all_s, ddof=1)
        return (mean_T1 - mean_T2) / std_dev if std_dev > 0 else 0

def execute_bias_analysis(model, tokenizer, results_path: str, hf_cache_dir: str, model_name: str, comments: str, languages: list):
    """Runs the layer-wise bias analysis and saves the results."""
    weathub_loader = WEATHubLoader(dataset_id='iamshnoo/WEATHub', cache_dir=os.path.join(hf_cache_dir, "datasets"))
    bias_quantifier = BiasQuantifier()
    num_layers = model.config.num_hidden_layers
    embedding_extractor = LayerEmbeddingExtractor(model, tokenizer)
    all_results = []
    weat_categories_to_test = ['WEAT1', 'WEAT2', 'WEAT6']

    for lang in languages:
        for weat_cat in weat_categories_to_test:
            print(f"\nProcessing: Lang='{lang}', Category='{weat_cat}'")
            word_lists = weathub_loader.get_word_lists(lang, weat_cat)
            if not word_lists: continue
            for layer_idx in tqdm(range(num_layers), desc=f"Layer Analysis ({lang}/{weat_cat})"):
                t1_embeds = embedding_extractor.get_embeddings(word_lists['targ1'], layer_idx)
                t2_embeds = embedding_extractor.get_embeddings(word_lists['targ2'], layer_idx)
                a1_embeds = embedding_extractor.get_embeddings(word_lists['attr1'], layer_idx)
                a2_embeds = embedding_extractor.get_embeddings(word_lists['attr2'], layer_idx)
                weat_score = bias_quantifier.weat_effect_size(t1_embeds, t2_embeds, a1_embeds, a2_embeds)
                all_results.append({'model_id': model_name, 'language': lang, 'weat_category_id': weat_cat, 'layer_idx': layer_idx, 'weat_score': weat_score, 'comments': comments})

    if all_results:
        results_df = pd.DataFrame(all_results)
        filename = f"bias_results_{model_name.replace('/', '_')}_{comments.replace(' ', '_')}.csv"
        filepath = os.path.join(results_path, filename)
        results_df.to_csv(filepath, index=False)
        print(f"Results successfully saved to: {filepath}")
    else:
        print("No results were generated.")
    print("\nAnalysis complete.")

# =============================================================================
# FINE-TUNING AND UPLOAD COMPONENTS
# =============================================================================

def create_prompt(example):
    """Creates a formatted instruction prompt from a dataset example."""
    template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:
{output}"""
    return template.format(instruction=example["instruction"], output=example['output'] + "</s>")

class BiasAnalysisCallback(TrainerCallback):
    """A custom TrainerCallback that runs bias analysis at the end of each epoch."""
    def __init__(self, tokenizer, results_path, hf_cache_dir, model_name):
        self.tokenizer = tokenizer
        self.results_path = results_path
        self.hf_cache_dir = hf_cache_dir
        self.model_name = model_name

    def on_epoch_end(self, args, state, control, **kwargs):
        epoch = int(state.epoch)
        model = kwargs['model']
        print(f"\n--- Running Bias Analysis for Epoch {epoch} ---")
        execute_bias_analysis(model, self.tokenizer, self.results_path, self.hf_cache_dir, self.model_name, f"After_epoch_{epoch}", ['en', 'hi'])
        print(f"--- Bias Analysis for Epoch {epoch} Completed ---")

def upload_to_hf(model_path, repo_name):
    """Uploads a model folder and associated artifacts to the Hugging Face Hub."""
    from huggingface_hub import HfApi, whoami
    
    print(f"Starting upload of '{model_path}' to '{repo_name}'...")
    
    # Quick authentication check
    try:
        user_info = whoami()
        current_user = user_info.get('name')
        print(f"✅ Authenticated as: {current_user}")
        
        # Verify the repo name matches the authenticated user
        expected_user = repo_name.split('/')[0]
        if current_user != expected_user:
            print(f"⚠️ WARNING: Authenticated user '{current_user}' doesn't match repo owner '{expected_user}'")
            repo_name = f"{current_user}/pythia-70m-finetuned-alpaca-hindi"
            print(f"🔄 Using corrected repo name: {repo_name}")
            
    except Exception as e:
        print(f"❌ Authentication check failed: {e}")
        print("Please run the HuggingFace login cell first!")
        return False
    
    # Check if model directory exists
    if not os.path.exists(model_path):
        print(f"❌ Model directory '{model_path}' not found!")
        return False
    
    # Attempt upload
    try:
        api = HfApi()
        
        # Create repository
        print(f"📝 Creating repository: {repo_name}")
        api.create_repo(repo_id=repo_name, repo_type="model", exist_ok=True)
        print("✅ Repository created/verified")
        
        # Upload folder
        print(f"📤 Uploading folder: {model_path}")
        api.upload_folder(
            folder_path=model_path, 
            repo_id=repo_name, 
            repo_type="model",
            commit_message="Upload fine-tuned Pythia-70m model for Hindi"
        )
        
        print("✅ Upload completed successfully!")
        print(f"🔗 View your model at: https://huggingface.co/{repo_name}")
        return True
        
    except Exception as e:
        print(f"❌ Upload failed with error: {e}")
        if "401" in str(e):
            print("🚨 401 Unauthorized Error - Please run the HuggingFace login cell first!")
        return False

# =============================================================================
# MAIN EXECUTION FUNCTION
# =============================================================================

def main(args):
    """Main function to orchestrate the fine-tuning and analysis process."""
    project_path, results_path, hf_cache_dir = setup_platform_environment(args.platform)

    if args.action == 'upload':
        if not os.path.exists(FINAL_MODEL_DIR):
            print(f"Error: Final model directory '{FINAL_MODEL_DIR}' not found. Please run training first.")
            return
        upload_to_hf(FINAL_MODEL_DIR, NEW_MODEL_REPO_NAME)
        return

    # --- Initial Bias Analysis (Before Fine-tuning) ---
    print("\n--- Running Initial Bias Analysis on Base Model ---")
    llm_manager = LLMManager(cache_dir=hf_cache_dir)
    base_model, base_tokenizer = llm_manager.load_model(BASE_MODEL_NAME, 'hf')
    if base_model and base_tokenizer:
        execute_bias_analysis(base_model, base_tokenizer, results_path, hf_cache_dir, BASE_MODEL_NAME, "Before_finetuning", ['en'])
    llm_manager.unload_model()
    print("--- Initial Bias Analysis Completed ---")

    # --- Fine-tuning ---
    print("\n--- Preparing for Fine-tuning ---")
    dataset = load_dataset(DATASET_NAME, split="train")
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    dataset_with_prompt = dataset.map(lambda example: {"text": create_prompt(example)})
    tokenized_dataset = dataset_with_prompt.map(lambda ex: tokenizer(ex["text"], truncation=True, max_length=512), batched=True, remove_columns=dataset.column_names)

    # Change below line for full and final execution.
    num_train_epochs = 1 if args.mode == 'test' else 5

    training_args = TrainingArguments(
        output_dir="./pythia-70m-hindi-tuned",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=8,
        learning_rate=2e-5,
        num_train_epochs=num_train_epochs,
        # max_steps=-1 if args.mode == 'full' else 50,
        max_steps=-1,
        bf16=True,
        logging_strategy="steps",
        logging_steps=10,
        save_strategy="epoch",
        report_to="none",
    )

    model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_NAME, torch_dtype=torch.bfloat16)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    if args.mode == 'test':
        shuffled_dataset = tokenized_dataset.shuffle(seed=42)
        train_dataset, eval_dataset = shuffled_dataset.select(range(100)), shuffled_dataset.select(range(100, 120))
    else:
        split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
        train_dataset, eval_dataset = split_dataset['train'], split_dataset['test']

    bias_analysis_callback = BiasAnalysisCallback(tokenizer, results_path, hf_cache_dir, BASE_MODEL_NAME)
    # trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset, data_collator=data_collator, tokenizer=tokenizer, callbacks=[bias_analysis_callback])

    trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset, data_collator=data_collator, processing_class=tokenizer, callbacks=[bias_analysis_callback])

    print(f"--- Starting Fine-tuning (Mode: {args.mode}) ---")
    trainer.train()
    print("--- Fine-tuning Completed ---")

    # Upload model to huggingface
    trainer.save_model(FINAL_MODEL_DIR)
    print(f"Final model saved to {FINAL_MODEL_DIR}")

    # Automatically upload to HuggingFace Hub
    print("\n--- Starting Automatic Upload to HuggingFace Hub ---")
    if os.path.exists(FINAL_MODEL_DIR):
        upload_to_hf(FINAL_MODEL_DIR, NEW_MODEL_REPO_NAME)
        print("--- Upload to HuggingFace Hub Completed ---")
    else:
        print(f"Error: Final model directory '{FINAL_MODEL_DIR}' not found.")


if __name__ == "__main__":
    is_notebook = 'google.colab' in sys.modules or 'ipykernel' in sys.modules

    if is_notebook:
        print("Running in a notebook environment. Setting arguments manually.")
        # Change below line for full and final execution.
        args = argparse.Namespace(action="train", mode="test", platform="digitalocean")
        main(args)
    else:
        parser = argparse.ArgumentParser(description="Fine-tune and analyze Pythia-70m.")
        parser.add_argument("--action", type=str, default="train", choices=["train", "upload"], help="Action to perform.")
        # Change below line for full and final execution.
        parser.add_argument("--mode", type=str, default="test", choices=["test", "full"], help="Training mode.")
        parser.add_argument("--platform", type=str, default="local", choices=["colab", "digitalocean", "local"], help="Execution platform.")
        parsed_args = parser.parse_args()
        main(parsed_args)


For Full Execution

In [ ]:
# -*- coding: utf-8 -*-
"""
This script merges the functionalities of fine-tuning a language model and performing
layer-wise bias analysis, including the final step of uploading the model to the
Hugging Face Hub. It is designed to be run on platforms like Google Colab or
DigitalOcean.

Workflow:
1.  **Initial Bias Analysis**: Performs a layer-wise bias analysis on the base
    model (`EleutherAI/pythia-70m`) using an English dataset before any fine-tuning.
2.  **Fine-tuning**: Fine-tunes the `EleutherAI/pythia-70m` model on a Hindi dataset
    (`iamshnoo/alpaca-cleaned-hindi`).
3.  **Post-Epoch Bias Analysis**: After each fine-tuning epoch, the script runs the
    layer-wise bias analysis on the model for both English and Hindi to track how
    bias evolves.
4.  **Results**: All bias analysis results are saved to CSV files for further examination.
5.  **Upload**: After training, the script can be run again with the 'upload' action
    to push the final model, tokenizer, and training artifacts to the Hugging Face Hub.

This script can be executed with command-line arguments to specify the action
('train' or 'upload'), training mode ('test' or 'full'), and the platform
('colab' or 'local').
"""

# In a notebook environment (like Colab), run this cell first to install dependencies.
# !pip install -q transformers>=4.32.0 datasets torch pandas numpy scikit-learn accelerate bitsandbytes

import os
import gc
import sys
import json
import csv
import argparse
import warnings
from tqdm import tqdm
import torch
import pandas as pd
import numpy as np
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    TrainerCallback,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from datasets import load_dataset
from huggingface_hub import HfApi, login
from sklearn.metrics.pairwise import cosine_similarity

# =============================================================================
# GLOBAL CONFIGURATION
# =============================================================================
BASE_MODEL_NAME = "EleutherAI/pythia-70m"
DATASET_NAME = "iamshnoo/alpaca-cleaned-hindi"
HF_USERNAME = "DebK"  # <-- IMPORTANT: SET YOUR USERNAME HERE
NEW_MODEL_REPO_NAME = f"{HF_USERNAME}/pythia-70m-finetuned-alpaca-hindi"
FINAL_MODEL_DIR = "./pythia-70m-hindi-final"
BIAS_RESULTS_DIR = "./bias_analysis_results"

# =============================================================================
# PLATFORM-SPECIFIC CONFIGURATION
# =============================================================================

def setup_platform_environment(platform: str = "local"):
    """
    Configures the environment based on the specified platform.

    Args:
        platform (str): The target platform, e.g., 'colab', 'digitalocean', 'local'.

    Returns:
        tuple: A tuple containing (project_path, results_path, hf_cache_dir).
    """
    print(f"Setting up environment for: {platform.upper()}")

    if platform == "colab":
        try:
            from google.colab import drive, userdata
            drive.mount('/content/drive')
            print("Google Drive mounted successfully.")

            project_path = "/content/drive/MyDrive/Mult_LLM_Bias/"
            results_path = os.path.join(project_path, "Results/Experiment1/")
            hf_cache_dir = os.path.join(project_path, "hf_cache/")

            print("Attempting to log in to Hugging Face...")
            HF_TOKEN = userdata.get('HF_TOKEN')
            if HF_TOKEN:
                login(HF_TOKEN, add_to_git_credential=True)
                print("Successfully logged in to Hugging Face!")
            else:
                print("Hugging Face token not found in Colab secrets.")

        except ImportError:
            print("Could not import Google Colab libraries. Defaulting to local setup.")
            return setup_platform_environment("local")

    elif platform == "digitalocean":
        print("Setting up DigitalOcean environment...")
        
        # Set paths for DigitalOcean - using root directory as base
        project_path = "/root/Mult_LLM_Bias/"
        results_path = os.path.join(project_path, "Results/Experiment1/")
        hf_cache_dir = os.path.join(project_path, "hf_cache/")
        
        # Check if block storage volume is available (optional)
        volume_path = "/mnt/volume_nyc1_01/"
        if os.path.exists(volume_path):
            print("DigitalOcean volume detected. Using volume storage...")
            project_path = os.path.join(volume_path, "Mult_LLM_Bias/")
            results_path = os.path.join(project_path, "Results/Experiment1/")
            hf_cache_dir = os.path.join(project_path, "hf_cache/")
        else:
            print("Using root directory storage...")
        
        # Set up Hugging Face authentication for DigitalOcean
        print("Setting up Hugging Face authentication...")
        
        # First try environment variable
        hf_token = os.environ.get('HF_TOKEN')
        if not hf_token:
            # Set your token
            hf_token = "HF_Secret"
            # Set it as environment variable for this session
            os.environ['HF_TOKEN'] = hf_token
        
        try:
            login(hf_token, add_to_git_credential=True)
            print("✅ Successfully logged in to Hugging Face!")
        except Exception as e:
            print(f"❌ Failed to login to Hugging Face: {e}")
            print("Please check your token and internet connection.")
        
        # Optimize for DigitalOcean environment
        os.environ['TRANSFORMERS_CACHE'] = hf_cache_dir
        os.environ['HF_HOME'] = hf_cache_dir
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Use first GPU if available
        
        print("DigitalOcean environment configured successfully.")

    else: # Default to local setup
        project_path = "./"
        results_path = BIAS_RESULTS_DIR
        hf_cache_dir = "./hf_cache/"

    os.makedirs(results_path, exist_ok=True)
    os.makedirs(hf_cache_dir, exist_ok=True)

    print(f"Project path set to: {project_path}")
    print(f"Bias analysis results will be saved to: {results_path}")

    return project_path, results_path, hf_cache_dir


# =============================================================================
# LAYER-WISE BIAS ANALYSIS COMPONENTS
# =============================================================================

class LLMManager:
    """Manages the lifecycle of LLMs to optimize memory usage."""
    def __init__(self, cache_dir: str):
        self.cache_dir = cache_dir
        self.model = None
        self.tokenizer = None
        self.current_model_id = None

    def load_model(self, model_id: str, model_repo: str):
        """Load model and tokenizer from either Hugging Face or a local path."""
        if self.current_model_id == model_id and self.model is not None:
            print(f"Model '{model_id}' already loaded.")
            return self.model, self.tokenizer

        print(f"Loading model: {model_id} from {model_repo}")
        load_path = model_id if model_repo == 'hf' else model_repo

        try:
            self.tokenizer = AutoTokenizer.from_pretrained(load_path, cache_dir=self.cache_dir)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            #Update Config
            quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16
             )

            # quantization_config = BitsAndBytesConfig(load_in_4bit=True)
            self.model = AutoModelForCausalLM.from_pretrained(
                load_path,
                torch_dtype=torch.float16,
                device_map="auto",
                quantization_config=quantization_config,
                cache_dir=self.cache_dir
            )
            self.current_model_id = model_id
            print(f"Model '{model_id}' loaded successfully.")
            return self.model, self.tokenizer

        except Exception as e:
            print(f"ERROR: Failed to load model '{model_id}'. Exception: {e}")
            return None, None

    def unload_model(self):
        """Unloads the model and clears GPU cache."""
        if self.model:
            print(f"Unloading model: {self.current_model_id}...")
            del self.model
            del self.tokenizer
            self.model, self.tokenizer, self.current_model_id = None, None, None
            gc.collect()
            torch.cuda.empty_cache()
            print("Model unloaded and memory cleared.")

class WEATHubLoader:
    """Loads the WEATHub dataset and provides word lists."""
    def __init__(self, dataset_id: str, cache_dir: str = None):
        print(f"Loading WEATHub dataset from '{dataset_id}'...")
        try:
            self.dataset = load_dataset(dataset_id, cache_dir=cache_dir)
            print("WEATHub dataset loaded successfully.")
            self.split_mapping = {
                'WEAT1': 'original_weat', 'WEAT2': 'original_weat', 'WEAT6': 'original_weat', 'WEAT7': 'original_weat', 'WEAT8': 'original_weat'
            }
        except Exception as e:
            print(f"ERROR: Failed to load WEATHub dataset. Exception: {e}")
            self.dataset = None

    def get_word_lists(self, language_code: str, weat_category_id: str):
        """Retrieves target and attribute word lists."""
        if not self.dataset: return None
        split_name = self.split_mapping.get(weat_category_id)
        if not split_name:
            print(f"Warning: Category '{weat_category_id}' not found.")
            return None
        try:
            filtered = self.dataset[split_name].filter(lambda x: x['language'] == language_code and x['weat'] == weat_category_id)
            if len(filtered) > 0:
                return { 'targ1': filtered[0]['targ1.examples'], 'targ2': filtered[0]['targ2.examples'], 'attr1': filtered[0]['attr1.examples'], 'attr2': filtered[0]['attr2.examples'] }
            else:
                print(f"Warning: No data for language '{language_code}' and category '{weat_category_id}'.")
                return None
        except Exception as e:
            print(f"Error filtering data for '{weat_category_id}' in language '{language_code}': {e}")
            return None

class LayerEmbeddingExtractor:
    """Extracts hidden states from model layers."""
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.device = model.device

    @torch.no_grad()
    def get_embeddings(self, words: list, layer_idx: int):
        """Gets embeddings for a list of words at a specific layer."""
        all_embeddings = []
        for word in words:
            inputs = self.tokenizer(word, return_tensors="pt", add_special_tokens=False).to(self.device)
            outputs = self.model(**inputs, output_hidden_states=True)
            word_embedding = outputs.hidden_states[layer_idx][0].mean(dim=0).float().cpu().numpy()
            all_embeddings.append(word_embedding)
        return np.array(all_embeddings)

class BiasQuantifier:
    """Calculates bias scores using WEAT effect size."""
    def _s(self, w, A, B):
        mean_cos_A = np.mean([cosine_similarity([w], [a])[0][0] for a in A])
        mean_cos_B = np.mean([cosine_similarity([w], [b])[0][0] for b in B])
        return mean_cos_A - mean_cos_B

    def weat_effect_size(self, T1_embeds, T2_embeds, A1_embeds, A2_embeds):
        """Calculates the WEAT effect size (d-score)."""
        mean_T1 = np.mean([self._s(t, A1_embeds, A2_embeds) for t in T1_embeds])
        mean_T2 = np.mean([self._s(t, A1_embeds, A2_embeds) for t in T2_embeds])
        all_s = [self._s(t, A1_embeds, A2_embeds) for t in np.concatenate((T1_embeds, T2_embeds))]
        std_dev = np.std(all_s, ddof=1)
        return (mean_T1 - mean_T2) / std_dev if std_dev > 0 else 0

def execute_bias_analysis(model, tokenizer, results_path: str, hf_cache_dir: str, model_name: str, comments: str, languages: list):
    """Runs the layer-wise bias analysis and saves the results."""
    weathub_loader = WEATHubLoader(dataset_id='iamshnoo/WEATHub', cache_dir=os.path.join(hf_cache_dir, "datasets"))
    bias_quantifier = BiasQuantifier()
    num_layers = model.config.num_hidden_layers
    embedding_extractor = LayerEmbeddingExtractor(model, tokenizer)
    all_results = []
    weat_categories_to_test = ['WEAT1', 'WEAT2', 'WEAT6']

    for lang in languages:
        for weat_cat in weat_categories_to_test:
            print(f"\nProcessing: Lang='{lang}', Category='{weat_cat}'")
            word_lists = weathub_loader.get_word_lists(lang, weat_cat)
            if not word_lists: continue
            for layer_idx in tqdm(range(num_layers), desc=f"Layer Analysis ({lang}/{weat_cat})"):
                t1_embeds = embedding_extractor.get_embeddings(word_lists['targ1'], layer_idx)
                t2_embeds = embedding_extractor.get_embeddings(word_lists['targ2'], layer_idx)
                a1_embeds = embedding_extractor.get_embeddings(word_lists['attr1'], layer_idx)
                a2_embeds = embedding_extractor.get_embeddings(word_lists['attr2'], layer_idx)
                weat_score = bias_quantifier.weat_effect_size(t1_embeds, t2_embeds, a1_embeds, a2_embeds)
                all_results.append({'model_id': model_name, 'language': lang, 'weat_category_id': weat_cat, 'layer_idx': layer_idx, 'weat_score': weat_score, 'comments': comments})

    if all_results:
        results_df = pd.DataFrame(all_results)
        filename = f"bias_results_{model_name.replace('/', '_')}_{comments.replace(' ', '_')}.csv"
        filepath = os.path.join(results_path, filename)
        results_df.to_csv(filepath, index=False)
        print(f"Results successfully saved to: {filepath}")
    else:
        print("No results were generated.")
    print("\nAnalysis complete.")

# =============================================================================
# FINE-TUNING AND UPLOAD COMPONENTS
# =============================================================================

def create_prompt(example):
    """Creates a formatted instruction prompt from a dataset example."""
    template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:
{output}"""
    return template.format(instruction=example["instruction"], output=example['output'] + "</s>")

class BiasAnalysisCallback(TrainerCallback):
    """A custom TrainerCallback that runs bias analysis at the end of each epoch."""
    def __init__(self, tokenizer, results_path, hf_cache_dir, model_name):
        self.tokenizer = tokenizer
        self.results_path = results_path
        self.hf_cache_dir = hf_cache_dir
        self.model_name = model_name

    def on_epoch_end(self, args, state, control, **kwargs):
        epoch = int(state.epoch)
        model = kwargs['model']
        print(f"\n--- Running Bias Analysis for Epoch {epoch} ---")
        execute_bias_analysis(model, self.tokenizer, self.results_path, self.hf_cache_dir, self.model_name, f"After_epoch_{epoch}", ['en', 'hi'])
        print(f"--- Bias Analysis for Epoch {epoch} Completed ---")

def upload_to_hf(model_path, repo_name):
    """Uploads a model folder and associated artifacts to the Hugging Face Hub."""
    from huggingface_hub import HfApi, whoami
    
    print(f"Starting upload of '{model_path}' to '{repo_name}'...")
    
    # Quick authentication check
    try:
        user_info = whoami()
        current_user = user_info.get('name')
        print(f"✅ Authenticated as: {current_user}")
        
        # Verify the repo name matches the authenticated user
        expected_user = repo_name.split('/')[0]
        if current_user != expected_user:
            print(f"⚠️ WARNING: Authenticated user '{current_user}' doesn't match repo owner '{expected_user}'")
            repo_name = f"{current_user}/pythia-70m-finetuned-alpaca-hindi"
            print(f"🔄 Using corrected repo name: {repo_name}")
            
    except Exception as e:
        print(f"❌ Authentication check failed: {e}")
        print("Please run the HuggingFace login cell first!")
        return False
    
    # Check if model directory exists
    if not os.path.exists(model_path):
        print(f"❌ Model directory '{model_path}' not found!")
        return False
    
    # Attempt upload
    try:
        api = HfApi()
        
        # Create repository
        print(f"📝 Creating repository: {repo_name}")
        api.create_repo(repo_id=repo_name, repo_type="model", exist_ok=True)
        print("✅ Repository created/verified")
        
        # Upload folder
        print(f"📤 Uploading folder: {model_path}")
        api.upload_folder(
            folder_path=model_path, 
            repo_id=repo_name, 
            repo_type="model",
            commit_message="Upload fine-tuned Pythia-70m model for Hindi"
        )
        
        print("✅ Upload completed successfully!")
        print(f"🔗 View your model at: https://huggingface.co/{repo_name}")
        return True
        
    except Exception as e:
        print(f"❌ Upload failed with error: {e}")
        if "401" in str(e):
            print("🚨 401 Unauthorized Error - Please run the HuggingFace login cell first!")
        return False

# =============================================================================
# MAIN EXECUTION FUNCTION
# =============================================================================

def main(args):
    """Main function to orchestrate the fine-tuning and analysis process."""
    project_path, results_path, hf_cache_dir = setup_platform_environment(args.platform)

    if args.action == 'upload':
        if not os.path.exists(FINAL_MODEL_DIR):
            print(f"Error: Final model directory '{FINAL_MODEL_DIR}' not found. Please run training first.")
            return
        upload_to_hf(FINAL_MODEL_DIR, NEW_MODEL_REPO_NAME)
        return

    # --- Initial Bias Analysis (Before Fine-tuning) ---
    print("\n--- Running Initial Bias Analysis on Base Model ---")
    llm_manager = LLMManager(cache_dir=hf_cache_dir)
    base_model, base_tokenizer = llm_manager.load_model(BASE_MODEL_NAME, 'hf')
    if base_model and base_tokenizer:
        execute_bias_analysis(base_model, base_tokenizer, results_path, hf_cache_dir, BASE_MODEL_NAME, "Before_finetuning", ['en'])
    llm_manager.unload_model()
    print("--- Initial Bias Analysis Completed ---")

    # --- Fine-tuning ---
    print("\n--- Preparing for Fine-tuning ---")
    dataset = load_dataset(DATASET_NAME, split="train")
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    dataset_with_prompt = dataset.map(lambda example: {"text": create_prompt(example)})
    tokenized_dataset = dataset_with_prompt.map(lambda ex: tokenizer(ex["text"], truncation=True, max_length=512), batched=True, remove_columns=dataset.column_names)

    # Change below line for full and final execution.
    num_train_epochs = 1 if args.mode == 'test' else 5

    training_args = TrainingArguments(
        output_dir="./pythia-70m-hindi-tuned",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=8,
        learning_rate=2e-5,
        num_train_epochs=num_train_epochs,
        # max_steps=-1 if args.mode == 'full' else 50,
        max_steps=-1,
        bf16=True,
        logging_strategy="steps",
        logging_steps=10,
        save_strategy="epoch",
        report_to="none",
    )

    model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_NAME, torch_dtype=torch.bfloat16)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    if args.mode == 'test':
        shuffled_dataset = tokenized_dataset.shuffle(seed=42)
        train_dataset, eval_dataset = shuffled_dataset.select(range(100)), shuffled_dataset.select(range(100, 120))
    else:
        split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
        train_dataset, eval_dataset = split_dataset['train'], split_dataset['test']

    bias_analysis_callback = BiasAnalysisCallback(tokenizer, results_path, hf_cache_dir, BASE_MODEL_NAME)
    # trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset, data_collator=data_collator, tokenizer=tokenizer, callbacks=[bias_analysis_callback])

    trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset, data_collator=data_collator, processing_class=tokenizer, callbacks=[bias_analysis_callback])

    print(f"--- Starting Fine-tuning (Mode: {args.mode}) ---")
    trainer.train()
    print("--- Fine-tuning Completed ---")

    # Upload model to huggingface
    trainer.save_model(FINAL_MODEL_DIR)
    print(f"Final model saved to {FINAL_MODEL_DIR}")

    # Automatically upload to HuggingFace Hub
    print("\n--- Starting Automatic Upload to HuggingFace Hub ---")
    if os.path.exists(FINAL_MODEL_DIR):
        upload_to_hf(FINAL_MODEL_DIR, NEW_MODEL_REPO_NAME)
        print("--- Upload to HuggingFace Hub Completed ---")
    else:
        print(f"Error: Final model directory '{FINAL_MODEL_DIR}' not found.")


if __name__ == "__main__":
    is_notebook = 'google.colab' in sys.modules or 'ipykernel' in sys.modules

    if is_notebook:
        print("Running in a notebook environment. Setting arguments manually.")
        # Change below line for full and final execution.
        args = argparse.Namespace(action="train", mode="full", platform="digitalocean")
        main(args)
    else:
        parser = argparse.ArgumentParser(description="Fine-tune and analyze Pythia-70m.")
        parser.add_argument("--action", type=str, default="train", choices=["train", "upload"], help="Action to perform.")
        # Change below line for full and final execution.
        parser.add_argument("--mode", type=str, default="full", choices=["test", "full"], help="Training mode.")
        parser.add_argument("--platform", type=str, default="local", choices=["colab", "digitalocean", "local"], help="Execution platform.")
        parsed_args = parser.parse_args()
        main(parsed_args)


Huggingface Test

c:\Users\Debz\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


✅ Successfully logged in to HuggingFace!
✅ Logged in as: Debk
